In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import re
from datetime import datetime, timedelta, timezone

import requests
import pandas as pd

import ipywidgets as widgets
from IPython.display import display
from io import StringIO



# -------------------------
# 基本設定
# -------------------------

JST = timezone(timedelta(hours=9))

# 高層観測地点一覧（コード, 名称）
STATIONS = [
    ("47401", "稚内"),
    ("47412", "札幌"),
    ("47418", "釧路"),
    ("47582", "秋田"),
    ("47600", "輪島"),
    ("47646", "館野"),
    ("47678", "八丈島"),
    ("47741", "松江"),
    ("47778", "潮岬"),
    ("47807", "福岡"),
    ("47827", "鹿児島"),
    ("47909", "名瀬／本茶峠"),
    ("47918", "石垣島"),
    ("47945", "南大東島"),
    ("47971", "父島"),
    ("47991", "南鳥島"),
    ("89532", "昭和（南極）"),
]


# -------------------------
# データ取得まわり
# -------------------------

def build_candidate_datetimes(now_jst: datetime, days_back: int = 4):
    """最近数日の 21時 / 9時 を新しい順に列挙"""
    candidates = []
    for d in range(days_back + 1):
        date = now_jst.date() - timedelta(days=d)
        for hour in (21, 9):  # 新しい方から試す
            dt = datetime(date.year, date.month, date.day, hour, tzinfo=JST)
            if dt <= now_jst:
                candidates.append(dt)
    # 重複を除いて新しい順
    return sorted(set(candidates), reverse=True)


def fetch_uth_page(point: str, dt: datetime) -> str | None:
    """気温・湿度(daily_uth.php) の HTML を取得"""
    base_url = "https://www.data.jma.go.jp/stats/etrn/upper/view/daily_uth.php"
    params = {
        "year": dt.year,
        "month": dt.month,
        "day": dt.day,
        "hour": dt.hour,
        "atm": "",
        "point": point,
        "view": "",
    }
    try:
        resp = requests.get(base_url, params=params, timeout=10)
        resp.raise_for_status()
        return resp.text
    except Exception as e:
        print(f"HTTP取得エラー: {e}")
        return None


def parse_html_table_to_df(html: str):
    """
    daily_uth の HTML からテーブルを read_html で読み、
    [気圧hpa, 実高度, 気温℃, 相対湿度％] の DataFrame にする。
    """
    try:
        tables = pd.read_html(StringIO(html))
    except ValueError:
        return None

    target = None
    for t in tables:
        cols_join = "".join(str(c) for c in t.columns)
        if "気圧" in cols_join and "高度" in cols_join and "気温" in cols_join:
            target = t
            break

    if target is None:
        return None

    # MultiIndex などを避けるため、列名を文字列化
    target.columns = [str(c) for c in target.columns]
    cols = list(target.columns)

    def find_col(keyword):
        for i, c in enumerate(cols):
            if keyword in c:
                return i
        return None

    i_p = find_col("気圧")
    i_h = find_col("高度")
    i_t = find_col("気温")
    i_rh = find_col("相対湿度")

    if None in (i_p, i_h, i_t, i_rh):
        return None

    df = target.iloc[:, [i_p, i_h, i_t, i_rh]].copy()
    df.columns = ["気圧hpa", "実高度", "気温℃", "相対湿度％"]

    # 数値化
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna().reset_index(drop=True)
    df = df.sort_values("気圧hpa", ascending=False).reset_index(drop=True)
    return df


def get_latest_sounding_df(point: str):
    """
    指定地点の「利用可能な最新の 09/21 時」データを探して DataFrame を返す。
    見つかれば (df, 日時) を返す。なければ (None, None)。
    """
    now_jst = datetime.now(JST)
    candidates = build_candidate_datetimes(now_jst, days_back=4)

    for dt in candidates:
        html = fetch_uth_page(point, dt)
        if html is None:
            continue

        df = parse_html_table_to_df(html)
        if df is None or df.empty:
            continue

        return df, dt

    return None, None


# -------------------------
# Jupyter ウィジェット UI
# -------------------------

station_dropdown = widgets.Dropdown(
    options=[(f"{name} ({code})", code) for code, name in STATIONS],
    value="47778",  # デフォルト: 潮岬
    description="地点:",
    layout=widgets.Layout(width="350px")
)

run_button = widgets.Button(
    description="最新データを取得して保存",
    button_style="primary",
    layout=widgets.Layout(width="350px")
)

output = widgets.Output()


def on_run_clicked(b):
    with output:
        output.clear_output()

        point = station_dropdown.value
        name = next(n for c, n in STATIONS if c == point)

        print(f"{name} ({point}) の最新データを取得中…")

        df, dt = get_latest_sounding_df(point)
        if df is None or df.empty:
            print("データが見つかりませんでした。")
            return

        ts_label = dt.strftime("%Y%m%d_%H%M")
        filename = f"大気データ_{name}_{ts_label}.csv"

        df.to_csv(filename, index=False, encoding="utf-8-sig")

        print(f"CSV を保存しました: {filename}")
        print("先頭5行:")
        display(df.head())


run_button.on_click(on_run_clicked)

ui = widgets.VBox([station_dropdown, run_button, output])
display(ui)


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
duct_raytrace_Mgradient_with_widgets.py  (Excel/CSV + Emagram + Rays)

■ 入力ファイルフォーマット (Excel / CSV 共通):

    列1: 気圧 [hPa]
    列2: 実高度 [m]
    列3: 気温 [℃]
    列4: 相対湿度 [%]
    1 行目はヘッダ
    '///' などの非数値は自動的に NaN として除去する。

■ 図の構成

  上 ：エマグラム（Temperature / Dew point vs Pressure, logP）
  下左：気温 / 露点温度 vs 高度 [ft]
  下右：Ray paths （Range [NM] vs Height [ft]）

  下段 2 つは同じ Height[ft] スケールを共有する。
"""

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 単位変換
FT2M = 0.3048
M2FT = 1.0 / FT2M
NM2M = 1852.0
KM2M = 1000.0
M2KM = 1.0 / KM2M


# ============================================================================
#  大気プロファイル
# ============================================================================

class AtmosphereProfile:
    """
    高度 z_ft に対する N(z), M(z), αM(z) を提供するクラス。
    内部では z_m, z_km も保持しておく。
    """

    def __init__(self, z_ft, P_hPa, T_C, RH_percent):
        z_ft = np.asarray(z_ft, dtype=float)
        P_hPa = np.asarray(P_hPa, dtype=float)
        T_C = np.asarray(T_C, dtype=float)
        RH_percent = np.asarray(RH_percent, dtype=float)

        # 高度でソート
        idx = np.argsort(z_ft)
        self.z_ft = z_ft[idx]
        self.z_m = self.z_ft * FT2M
        self.z_km = self.z_m * M2KM
        self.P_hPa = P_hPa[idx]
        self.T_C = T_C[idx]
        self.RH = RH_percent[idx]

        # N, M, αM を事前計算
        self.N = self._compute_N()
        self.M, self.alphaM_km = self._compute_M_and_alpha()

    # ------------------------------------------------------------------    
    @staticmethod
    def _sat_vapor_pressure(T_C):
        """Tetens の式による飽和水蒸気圧 [hPa]。"""
        return 6.11 * 10.0 ** (7.5 * T_C / (237.3 + T_C))

    def _compute_N(self):
        """電波レフラクトビティ N(z) [N-units]。"""
        T_K = self.T_C + 273.15
        e_s = self._sat_vapor_pressure(self.T_C)  # 飽和水蒸気圧
        e = (self.RH / 100.0) * e_s               # 実際の水蒸気分圧

        N = 77.6 * self.P_hPa / T_K + 3.73e5 * e / (T_K ** 2)
        return N

    def _compute_M_and_alpha(self):
        """
        M(z) = N + 0.157 * z[m] を計算し、
        αM = dM/dz [M-units/km] を求める。
        """
        z_m = self.z_m

        M = self.N + 0.157 * z_m          # M-units
        dM_dz_m = np.gradient(M, z_m)     # M-units/m
        alphaM_km = dM_dz_m * 1000.0      # M-units/km

        return M, alphaM_km

    # ------------------------------------------------------------------    
    @staticmethod
    def _interp_linear_extrap(x, xp, yp):
        """
        xp, yp に対し、内部は線形補間、外側は両端の傾きで線形外挿。
        x はスカラーでも配列でも可。
        """
        xp = np.asarray(xp, dtype=float)
        yp = np.asarray(yp, dtype=float)

        x_arr = np.asarray(x, dtype=float)
        scalar_input = False
        if x_arr.ndim == 0:
            x_arr = x_arr.reshape(1)
            scalar_input = True

        # 通常の線形補間
        y = np.interp(x_arr, xp, yp)

        # 左側外挿
        if xp[1] != xp[0]:
            slope_left = (yp[1] - yp[0]) / (xp[1] - xp[0])
        else:
            slope_left = 0.0
        mask_left = x_arr < xp[0]
        if np.any(mask_left):
            y[mask_left] = yp[0] + slope_left * (x_arr[mask_left] - xp[0])

        # 右側外挿
        if xp[-1] != xp[-2]:
            slope_right = (yp[-1] - yp[-2]) / (xp[-1] - xp[-2])
        else:
            slope_right = 0.0
        mask_right = x_arr > xp[-1]
        if np.any(mask_right):
            y[mask_right] = yp[-1] + slope_right * (x_arr[mask_right] - xp[-1])

        if scalar_input:
            return float(y[0])
        return y

    # ------------------------------------------------------------------    
    def alphaM_of_h(self, h_km):
        """高さ h[km] の αM(h): 線形補間＋両端は線形外挿。"""
        return self._interp_linear_extrap(h_km, self.z_km, self.alphaM_km)

    def M_of_h(self, h_km):
        """高さ h[km] の M(h): 線形補間＋両端は線形外挿。"""
        return self._interp_linear_extrap(h_km, self.z_km, self.M)


# ============================================================================
#  露点温度など（エマグラム用）
# ============================================================================

def dewpoint_from_T_RH(T_C, RH):
    """気温[℃] と 相対湿度[%] から露点温度[℃]を求める（Tetens 近似）。"""
    T = np.asarray(T_C, dtype=float)
    RH = np.asarray(RH, dtype=float)

    es = 6.112 * np.exp((17.67 * T) / (T + 243.5))   # 飽和水蒸気圧 [hPa]
    e  = RH / 100.0 * es                             # 実際の水蒸気圧 [hPa]

    ln_e_6 = np.log(e / 6.112)
    Td = (243.5 * ln_e_6) / (17.67 - ln_e_6)
    return Td


# ============================================================================
#  ファイル読み込み (Excel / CSV)
# ============================================================================

def load_profile_from_file(path, sheet_name=0):
    """
    Excel(.xlsx/.xls) または CSV(.csv) から大気プロファイルを読み込む。

    戻り値:
        atm       : AtmosphereProfile インスタンス
        P_hPa     : 1D array [hPa]
        T_C       : 1D array [℃]
        RH        : 1D array [%]
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"ファイルが見つかりません: {path}")

    _, ext = os.path.splitext(path)
    ext = ext.lower()

    # --- ファイル形式ごとの読み込み --------------------------------------
    if ext == ".csv":
        df = pd.read_csv(path, header=0)
    elif ext in (".xlsx", ".xls"):
        engine = "openpyxl" if ext == ".xlsx" else None
        df = pd.read_excel(path, sheet_name=sheet_name, header=0, engine=engine)
    else:
        raise ValueError("対応形式は .xlsx / .xls / .csv のみです。")

    cols = df.columns[:4]
    # '///' などを NaN にしてから削除
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna(subset=cols, how="any")
    if df.shape[0] < 2:
        raise ValueError("有効なデータ行が 2 行未満です。ファイル内容を確認してください。")

    P_hPa = df.iloc[:, 0].to_numpy(float)
    z_m   = df.iloc[:, 1].to_numpy(float)
    T_C   = df.iloc[:, 2].to_numpy(float)
    RH    = df.iloc[:, 3].to_numpy(float)

    # m → ft に変換し、0ft 以上のみ使用（上側は全データ残す）
    z_ft = z_m * M2FT
    mask = (z_ft >= 0.0)
    if not np.any(mask):
        raise ValueError("0 ft 以上のデータがありません。")

    P_use = P_hPa[mask]
    z_ft_use = z_ft[mask]
    T_use = T_C[mask]
    RH_use = RH[mask]

    atm = AtmosphereProfile(z_ft_use, P_use, T_use, RH_use)

    # エマグラム用は、「元の順番」から 0ft 以上だけ抜いた P,T,RH を返す
    return atm, P_use, T_use, RH_use


# ============================================================================
#  M 勾配にもとづくレイトレース
# ============================================================================

def trace_ray_Mgradient(
    atm: AtmosphereProfile,
    h0_ft: float,
    theta0_deg: float,
    x_max_nm: float = 300.0,
    ds_km: float = 0.1,
    max_height_ft: float = 30000.0,
):
    """
    M 勾配 αM(h) に基づくレイを 1 本トレースする。
    """
    h0_km = h0_ft * FT2M * M2KM
    x_max_km = x_max_nm * NM2M * M2KM
    max_h_km = max_height_ft * FT2M * M2KM

    # 安全な最大ステップ数
    n_steps = int(x_max_km / ds_km) + 5

    x_km = np.zeros(n_steps)
    h_km = np.zeros(n_steps)
    theta = np.zeros(n_steps)

    x_km[0] = 0.0
    h_km[0] = h0_km
    theta[0] = np.deg2rad(theta0_deg)

    for i in range(1, n_steps):
        x_prev = x_km[i-1]
        h_prev = h_km[i-1]
        th_prev = theta[i-1]

        # 高度が範囲外なら終了
        if h_prev < 0.0 or h_prev > max_h_km:
            x_km = x_km[:i]
            h_km = h_km[:i]
            theta = theta[:i]
            break

        alphaM = atm.alphaM_of_h(h_prev)  # [M-units/km]

        # θ 更新: dθ/ds = -10^-6 * αM
        th_new = th_prev - alphaM * ds_km * 1e-6

        # h 更新: MATLAB コードと同じ形
        if abs(alphaM) > 1e-6:
            h_new = h_prev + ((th_new**2 - th_prev**2) * 1e6) / (2.0 * alphaM)
        else:
            # αM ≈ 0 のときはほぼ直線
            h_new = h_prev + th_prev * ds_km

        x_new = x_prev + ds_km

        x_km[i] = x_new
        h_km[i] = h_new
        theta[i] = th_new

        if x_new >= x_max_km:
            x_km = x_km[:i+1]
            h_km = h_km[:i+1]
            theta = theta[:i+1]
            break
        if h_new < 0.0 or h_new > max_h_km:
            x_km = x_km[:i+1]
            h_km = h_km[:i+1]
            theta = theta[:i+1]
            break

    # 単位変換
    x_nm = x_km * KM2M / NM2M
    h_ft = h_km * KM2M * M2FT
    return x_nm, h_ft


# ============================================================================
#  描画：エマグラム + Ray paths（上 1 段＋下 2 枚）
# ============================================================================

def plot_emagram_and_rays(atm: AtmosphereProfile,
                          P_hPa, T_C, RH,
                          src_height_ft: float,
                          angles_deg,
                          x_max_nm: float = 300.0,
                          max_height_ft: float = 30000.0,
                          filename: str | None = None,
                          show: bool = True):
    """
    上段：エマグラム（T, Td vs P）
    下左：T, Td vs Height
    下右：Ray paths (Range vs Height)

    filename が None のときは PNG を保存しない。
    """

    # --- 露点温度 ---
    Td_C = dewpoint_from_T_RH(T_C, RH)

    # ---------------------------------------------------------
    # 上段用：P 軸のエマグラム（logP）
    # ---------------------------------------------------------
    # 気圧は上空ほど小さいので昇順ソート（描画時に反転）
    sortP = np.argsort(P_hPa)
    P = P_hPa[sortP]
    T_P = T_C[sortP]
    Td_P = Td_C[sortP]

    # 300 hPa より上空(P <= 300 hPa)は露点を描かない
    Td_P_plot = Td_P.copy()
    Td_P_plot[P <= 300.0] = np.nan

    # ---------------------------------------------------------
    # 下段左用：高度軸の T / Td
    # AtmosphereProfile 内部は高度で昇順ソート済み
    # ---------------------------------------------------------
    z_ft_prof = atm.z_ft.copy()
    T_H = atm.T_C.copy()
    RH_H = atm.RH.copy()
    P_H = atm.P_hPa.copy()
    Td_H = dewpoint_from_T_RH(T_H, RH_H)

    Td_H_plot = Td_H.copy()
    Td_H_plot[P_H <= 300.0] = np.nan

    # ---------------------------------------------------------
    # レイパスを事前計算
    # ---------------------------------------------------------
    rays = []
    for th in angles_deg:
        x_nm, h_ft = trace_ray_Mgradient(
            atm,
            h0_ft=src_height_ft,
            theta0_deg=th,
            x_max_nm=x_max_nm,
            ds_km=0.1,
            max_height_ft=max_height_ft,
        )
        rays.append((th, x_nm, h_ft))

    # ---------------------------------------------------------
    # 図レイアウト：2 行 2 列
    #   上  : [0, :]  → ax_top
    #   下左: [1, 0]  → ax_bl
    #   下右: [1, 1]  → ax_br（下段は同じ Height[ft] スケール）
    # ---------------------------------------------------------
    fig = plt.figure(figsize=(10, 10))
    gs = fig.add_gridspec(
        2, 2,
        height_ratios=[1, 1],
        width_ratios=[0.1, 0.9]
    )

    ax_top = fig.add_subplot(gs[0, :])
    ax_bl  = fig.add_subplot(gs[1, 0])
    ax_br  = fig.add_subplot(gs[1, 1], sharey=ax_bl)

    # ---------------------------------------------------------
    # 上段：エマグラム（背景の補助線 → 実測プロット）
    # ---------------------------------------------------------

    # 背景用の圧力座標（上端: 高圧, 下端: 低圧）
    P_bg = np.linspace(P.max(), P.min(), 60)

    # 1) 等温線（縦線）: -40, -30, ..., +40 ℃
    for T_iso in range(-40, 41, 10):
        ax_top.plot(
            np.full_like(P_bg, T_iso),
            P_bg,
            color="0.8",
            linestyle="--",
            linewidth=0.5,
            zorder=0,
        )

    # 2) 乾燥断熱線（dry adiabats）
    p0 = 1000.0          # 基準圧 [hPa]
    k  = 0.286           # R_d / c_p の近似値
    theta_vals = range(250, 361, 10)  # 250～360 K を 10K 間隔で

    for theta in theta_vals:
        T_K = theta * (P_bg / p0) ** k
        T_line = T_K - 273.15  # [℃]
        ax_top.plot(
            T_line,
            P_bg,
            color="0.8",
            linestyle="-",
            linewidth=0.5,
            zorder=0,
        )

    # 実測 T, Td を上書き
    ax_top.plot(T_P,       P, color="tab:red",  label="Temperature", zorder=2)
    ax_top.plot(Td_P_plot, P, color="tab:blue", label="Dew point",   zorder=2)

    ax_top.set_xlabel("Temperature [°C]")
    ax_top.set_ylabel("Pressure [hPa]")
    ax_top.set_yscale("log")
    ax_top.set_ylim(P.max(), P.min())  # 上空が上
    ax_top.yaxis.set_major_formatter(plt.ScalarFormatter())
    ax_top.grid(True, which="both", linestyle="--", linewidth=0.5, zorder=1)
    ax_top.legend(loc="best")
    ax_top.set_title("Emagram (T / Td vs Pressure)")

    # ---------------------------------------------------------
    # 下左：T / Td vs Height
    # ---------------------------------------------------------
    ax_bl.plot(T_H,       z_ft_prof, color="tab:red",  label="Temperature")
    ax_bl.plot(Td_H_plot, z_ft_prof, color="tab:blue", label="Dew point")

    ax_bl.set_xlabel("Temperature [°C]")
    ax_bl.set_ylabel("Height [ft]")
    ax_bl.set_ylim(0.0, max_height_ft)
    ax_bl.grid(True, which="both", linestyle="--", linewidth=0.5)
    ax_bl.legend(loc="best")
    ax_bl.set_title("T / Td vs Height")

    # ---------------------------------------------------------
    # 下右：Ray paths（下左と Height[ft] を共有）
    # ---------------------------------------------------------
    for th, x_nm, h_ft in rays:
        ax_br.plot(x_nm, h_ft)
    ax_br.set_xlabel("Range [NM]")
    ax_br.set_xlim(0.0, x_max_nm)
    ax_br.set_ylim(0.0, max_height_ft)
    ax_br.grid(True, which="both", linestyle="--", linewidth=0.5)
    ax_br.set_title("Ray paths (M-gradient)")

    fig.tight_layout()

    # PNG 保存は任意
    if filename is not None:
        fig.savefig(filename, dpi=150, bbox_inches="tight")

    if show:
        plt.show()
    plt.close(fig)

    if filename is not None:
        print("Saved:", filename)


# ============================================================================
#  実行関数
# ============================================================================

def run_raytrace_with_params(file_path: str,
                             src_height_ft: float,
                             angle_start_deg: float,
                             angle_end_deg: float,
                             n_rays: int,
                             x_max_nm: float,
                             max_height_ft: float):
    """
    パラメータ指定でレイトレースを実行して図を描く。

    angle_start_deg : 開始角度 [deg]（水平から仰角は正、俯角は負）
    angle_end_deg   : 終了角度 [deg]（同上）
    例:
      -1 ～ -5 度 : angle_start_deg=-1, angle_end_deg=-5
       3 ～ -5 度 : angle_start_deg= 3, angle_end_deg=-5
    """
    atm, P, T, RH = load_profile_from_file(file_path)

    # 角度列：開始角度 ～ 終了角度 を等間隔
    if n_rays < 2:
        angles = np.array([-angle_start_deg])
    else:
        angles = np.linspace(-angle_start_deg, -angle_end_deg, n_rays)

    print(f"File: {file_path}")
    print(f"Source height: {src_height_ft:.1f} ft")
    print(f"Angle start: {angle_start_deg:.1f} deg, Angle end: {angle_end_deg:.1f} deg")
    print(f"Rays: {n_rays}, Max range: {x_max_nm:.1f} NM, Max height: {max_height_ft:.1f} ft")

    plot_emagram_and_rays(
        atm,
        P_hPa=P,
        T_C=T,
        RH=RH,
        src_height_ft=src_height_ft,
        angles_deg=angles,
        x_max_nm=x_max_nm,
        max_height_ft=max_height_ft,
        filename=None,  # ← PNG を毎回保存しない
        show=True,
    )


# ============================================================================
#  ウィジェット UI
# ============================================================================

def launch_widgets(default_file: str = "大気データ.xlsx"):
    """
    Jupyter / IPython 上で、パラメータをウィジェットから指定できる UI を出す。
    ipywidgets が無い環境では input() フォールバックに切り替わる。
    """
    try:
        import ipywidgets as widgets
        from IPython.display import display
    except Exception:
        # フォールバック: コンソール入力
        print("ipywidgets が利用できません。コンソール入力モードで実行します。")
        file_path = input(f"ファイル名 ({default_file} など): ").strip() or default_file
        src_h = float(input("Source height [ft] (例: 1500): "))
        ang_start = float(input("Start angle [deg] (例: 3): "))
        ang_end   = float(input("End angle [deg] (例: -5): "))
        n    = int(input("Rays (例: 100): "))
        rmax = float(input("Max range [NM] (例: 100): "))
        hmax = float(input("Max height [ft] (例: 30000): "))
        run_raytrace_with_params(file_path, src_h, ang_start, ang_end, n, rmax, hmax)
        return

    # ファイル一覧 (xlsx / xls / csv) を取得して Dropdown 用 options にする
    files_xlsx = glob.glob("*.xlsx")
    files_xls  = glob.glob("*.xls")
    files_csv  = glob.glob("*.csv")
    data_files = sorted(set(files_xlsx + files_xls + files_csv))

    if not data_files:
        data_files = [default_file]

    default_value = default_file if default_file in data_files else data_files[0]

    file_dropdown = widgets.Dropdown(
        options=data_files,
        value=default_value,
        description="File:",
        layout=widgets.Layout(width="300px"),
    )

    src_height = widgets.FloatText(
        value=1500.0,
        description="Src [ft]:",
        step=100.0,
    )

    max_height = widgets.FloatText(
        value=30000.0,
        description="Max H [ft]:",
    )

    max_range = widgets.FloatText(
        value=500.0,
        description="Max R [NM]:",
    )

    # 開始・終了角度（符号付きで直接指定）
    angle_start = widgets.FloatText(
        value=3.0,
        description="Start [deg]:",
    )

    angle_end = widgets.FloatText(
        value=-5.0,
        description="End [deg]:",
    )

    n_rays = widgets.IntText(
        value=100,
        description="Rays:",
    )

    run_button = widgets.Button(
        description="実行",
        button_style="primary",
    )

    output = widgets.Output(layout={"border": "1px solid #ccc"})

    def on_run_clicked(_):
        output.clear_output()
        with output:
            file_path = file_dropdown.value
            try:
                run_raytrace_with_params(
                    file_path=file_path,
                    src_height_ft=src_height.value,
                    angle_start_deg=angle_start.value,
                    angle_end_deg=angle_end.value,
                    n_rays=n_rays.value,
                    x_max_nm=max_range.value,
                    max_height_ft=max_height.value,
                )
            except Exception as e:
                print("エラーが発生しました:")
                print(repr(e))

    run_button.on_click(on_run_clicked)

    ui = widgets.VBox([
        file_dropdown,
        widgets.HBox([src_height, max_height, max_range]),
        widgets.HBox([angle_start, angle_end, n_rays]),
        run_button,
        output,
    ])
    display(ui)


# ============================================================================
#  メイン
# ============================================================================

if __name__ == "__main__":
    launch_widgets(default_file="大気データ.xlsx")


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Wyoming 大気鉛直プロファイル取得・CSV 保存ツール（RELH 不使用版）

- University of Wyoming sounding テキスト (TYPE=TEXT:LIST) から
  PRES[hPa], HGHT[m], TEMP[C], DWPT[C] を取得し、
  レイトレース／エマグラム用フォーマット

    気圧hpa, 実高度, 気温℃, 露点温度℃

  で CSV に保存する。

- ウィジェット上で観測所をプルダウン選択して
  「Download latest sounding」を押すだけ。

- まず現在時刻の直近 00Z / 12Z を試し、
  ダメなら 12 時間ずつ最大 3 サイクル（36 時間）まで遡って探す。
"""

import re
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

# ==== 観測所リスト ==========================================================
STATIONS = {
    "Sapporo (札幌) 47412": "47412",
    "Wajima (輪島) 47600": "47600",
    "Tateno/Tsukuba (館野) 47646": "47646",
    "Tokyo (東京) 47682": "47682",
    "Shionomisaki (潮岬) 47778": "47778",
    "Fukuoka (福岡) 47807": "47807",
    "Kagoshima (鹿児島) 47827": "47827",
    "Naha (那覇) 47936": "47936",
}

# ==== Wyoming サイトからテキストを取得 =====================================

def build_wy_url(station_id: str, dt_utc: datetime) -> str:
    """University of Wyoming sounding テキスト URL を作る。"""
    year  = dt_utc.year
    month = dt_utc.month
    day   = dt_utc.day
    hour  = dt_utc.hour     # 0 or 12 のはず

    ddhh = f"{day:02d}{hour:02d}"  # FROM, TO は DDHH 形式
    url = (
        "http://weather.uwyo.edu/cgi-bin/sounding"
        f"?region=asia&TYPE=TEXT:LIST"
        f"&YEAR={year}&MONTH={month:02d}&FROM={ddhh}&TO={ddhh}&STNM={station_id}"
    )
    return url


def fetch_sounding_text(station_id: str, dt_utc: datetime) -> str:
    """指定時刻 (UTC) のサウンディングテキストを取得。"""
    url = build_wy_url(station_id, dt_utc)
    resp = requests.get(url, timeout=20)
    resp.raise_for_status()
    return resp.text


# ==== テキスト → DataFrame 変換（PRES/HGHT/TEMP/DWPT のみ） =================

def parse_sounding_text(text: str) -> pd.DataFrame:
    """
    Wyoming の TYPE=TEXT:LIST テキストから
    PRES, HGHT, TEMP, DWPT の 4 列だけを抜き出して DataFrame にする。

    ヘッダ行 'PRES   HGHT   TEMP   DWPT ...' を探し、その直後から
    空行 or 'Station information' などが出るまで読む。
    """
    lines = text.splitlines()

    header_idx = None
    for i, line in enumerate(lines):
        if re.search(r"\bPRES\b.*\bHGHT\b.*\bTEMP\b.*\bDWPT\b", line):
            header_idx = i
            break

    if header_idx is None:
        raise ValueError("Cannot find data header line in sounding text.")

    data_rows = []
    for line in lines[header_idx + 1 :]:
        line_stripped = line.strip()
        if not line_stripped:
            break
        if line_stripped.startswith("Station information"):
            break
        if line_stripped.startswith("OBSERVED") or line_stripped.startswith("Forecast"):
            break

        # 数値行だけを対象にする
        if not re.match(r"^[0-9\-]", line_stripped):
            continue

        parts = line_stripped.split()
        # PRES HGHT TEMP DWPT RELH MIXR ...
        if len(parts) < 4:
            continue

        try:
            pres = float(parts[0])  # PRES [hPa]
            hght = float(parts[1])  # HGHT [m]
            temp = float(parts[2])  # TEMP [C]
            dwpt = float(parts[3])  # DWPT [C]
        except ValueError:
            continue

        data_rows.append((pres, hght, temp, dwpt))

    if not data_rows:
        raise ValueError("No data rows parsed in sounding text.")

    df = pd.DataFrame(
        data_rows,
        columns=["PRES_hPa", "HGHT_m", "TEMP_C", "DWPT_C"]
    )
    return df


# ==== 「最新 00Z/12Z を探す」ラッパ =========================================

def fetch_latest_sounding(station_id: str, dt_now_utc: datetime | None = None):
    """
    station_id について、今から最大 3 サイクル (36h) 遡りながら
    00Z / 12Z のサウンディングを探して DataFrame を返す。

    戻り値: (df, used_datetime_utc)
    """
    if dt_now_utc is None:
        dt_now_utc = datetime.now(timezone.utc)

    base = dt_now_utc.replace(minute=0, second=0, microsecond=0)
    base = base.replace(hour=12 if base.hour >= 12 else 0)

    last_error: Exception | None = None

    for k in range(4):  # 0,1,2,3 サイクルぶん試す
        dt_try = base - timedelta(hours=12 * k)
        try:
            txt = fetch_sounding_text(station_id, dt_try)
            df = parse_sounding_text(txt)
            return df, dt_try
        except Exception as e:
            last_error = e
            continue

    raise ValueError(f"Failed to fetch/parse latest sounding. Last error: {last_error!r}")


# ==== CSV へ保存（Wyoming 露点付きフォーマット） ============================

def save_to_csv_for_raytrace(df_raw: pd.DataFrame,
                             station_label: str,
                             dt_used_utc: datetime) -> str:
    """
    取得した df_raw (PRES_hPa, HGHT_m, TEMP_C, DWPT_C) を、
    4 列 CSV に変換して保存する。

    ファイル名: 'wyoming_<station>_<YYYYMMDD>_<HH>Z.csv'
    """
    safe_label = re.sub(r"[^\w\-]+", "_", station_label).strip("_")

    ymd = dt_used_utc.strftime("%Y%m%d")
    hh  = dt_used_utc.strftime("%H")
    fname = f"wyoming_{safe_label}_{ymd}_{hh}Z.csv"

    df = pd.DataFrame({
        "気圧hpa":    df_raw["PRES_hPa"].values,
        "実高度":     df_raw["HGHT_m"].values,
        "気温℃":      df_raw["TEMP_C"].values,
        "露点温度℃": df_raw["DWPT_C"].values,
    })
    df.to_csv(fname, index=False, encoding="utf-8-sig")
    return fname


# ==== ウィジェット UI =======================================================

def launch_widgets():
    """Jupyter 用ウィジェット UI"""
    try:
        import ipywidgets as widgets
        from IPython.display import display
    except ImportError:
        print("ipywidgets が必要です。`pip install ipywidgets` を実行してください。")
        return

    station_dd = widgets.Dropdown(
        options=list(STATIONS.keys()),
        description="Station:",
        layout=widgets.Layout(width="350px"),
    )
    run_btn = widgets.Button(description="Download latest sounding")
    out = widgets.Output()

    def on_click(_):
        with out:
            out.clear_output()
            st_label = station_dd.value
            st_id = STATIONS[st_label]
            print(f"Download latest for: {st_label} (ID={st_id})")

            try:
                df_raw, dt_used = fetch_latest_sounding(st_id)
                fname = save_to_csv_for_raytrace(df_raw, st_label, dt_used)
                print("Succeeded.")
                print("  Date/Time (UTC):", dt_used.strftime("%Y-%m-%d %H:%M"))
                print("  Saved CSV:", fname)
                print()
                print("Head of data:")
                display(df_raw.head())
            except Exception as e:
                print("Failed to get sounding.")
                print("  Reason:", repr(e))

    run_btn.on_click(on_click)

    ui = widgets.VBox([station_dd, run_btn, out])
    display(ui)


# ==== スクリプトとして実行されたとき ========================================

if __name__ == "__main__":
    try:
        launch_widgets()
    except Exception:
        # コンソール fallback
        print("Widget UI が使えない環境なので、コンソールモードで実行します。")
        print("使用可能な station:")
        for i, name in enumerate(STATIONS.keys(), 1):
            print(f"  [{i}] {name}")
        idx = int(input("番号を選択: ")) - 1
        st_label = list(STATIONS.keys())[idx]
        st_id = STATIONS[st_label]
        print(f"Selected: {st_label} ({st_id})")

        df_raw, dt_used = fetch_latest_sounding(st_id)
        fname = save_to_csv_for_raytrace(df_raw, st_label, dt_used)
        print("Saved:", fname)

In [35]:
# 依存が無ければ:
# !pip install requests beautifulsoup4 pandas lxml ipywidgets

import re, io
from urllib.parse import urljoin, parse_qs, urlparse
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import requests
import pandas as pd
from bs4 import BeautifulSoup
import ipywidgets as W
from IPython.display import display, clear_output

BASE = "https://www.data.jma.go.jp/stats/etrn/upper/"
INDEX_URL = urljoin(BASE, "index.php")
# 探索順（JST）: 遅い時刻から確認
CANDIDATE_HOURS = [21, 18, 15, 12, 9, 6, 3, 0]

# -------------- 共通 --------------
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (compatible; jma-upper-jupyter/recent)",
    "Accept-Language": "ja,en;q=0.8",
    "Referer": INDEX_URL,
})

def _fix_encoding(resp: requests.Response) -> str:
    if not resp.encoding or resp.encoding.lower() in ("iso-8859-1", "ascii"):
        resp.encoding = resp.apparent_encoding or "utf-8"
    return resp.text

def _soup(html: str) -> BeautifulSoup:
    try:
        return BeautifulSoup(html, "lxml")
    except Exception:
        return BeautifulSoup(html, "html.parser")

# -------------- 地点プルダウン --------------
def fetch_station_list():
    r = session.get(INDEX_URL, params={"year":"", "month":"", "day":"", "hour":"", "atm":"", "point":""}, timeout=20)
    html = _fix_encoding(r)
    s = _soup(html)
    opts = []
    for a in s.find_all("a", href=True):
        href = a["href"]
        if "point=" not in href:
            continue
        full = urljoin(r.url, href)
        qs = parse_qs(urlparse(full).query)
        point = (qs.get("point") or [""])[0]
        if re.fullmatch(r"\d{5}", point):
            label = (a.get_text() or "").strip()
            if label:
                opts.append((f"{label} (point={point})", point))
    # 重複排除 & コード順
    seen, out = set(), []
    for lab, p in opts:
        if p not in seen:
            seen.add(p); out.append((lab, p))
    out.sort(key=lambda x: x[1])
    if not out:
        raise RuntimeError("地点リンクが取得できませんでした。")
    return out

# -------------- 1回分の取得 --------------
def _read_biggest_table(html: str):
    try:
        tables = pd.read_html(io.StringIO(html))  # FutureWarning回避
    except ValueError:
        return None
    if not tables:
        return None
    df = max(tables, key=lambda d: d.shape[0] * d.shape[1])
    if df.shape[0] < 3 or df.shape[1] < 2:
        return None
    return df

def fetch_one(point: str, y: int, m: int, d: int, h: int):
    params = {"year": y, "month": m, "day": d, "hour": h, "atm": "", "point": point}

    # 1) 指定気圧面（hourly_usp.php）
    r1 = session.get(urljoin(BASE, "view/hourly_usp.php"), params=params, timeout=20)
    if r1.ok:
        html1 = _fix_encoding(r1)
        df1 = _read_biggest_table(html1)
        if df1 is not None:
            df1.attrs["source"] = "hourly_usp"
            return df1

    # 2) 気温・湿度（daily_uth.php）
    r2 = session.get(urljoin(BASE, "view/daily_uth.php"), params=params, timeout=20)
    if r2.ok:
        html2 = _fix_encoding(r2)
        df2 = _read_biggest_table(html2)
        if df2 is not None:
            df2.attrs["source"] = "daily_uth"
            return df2

    return None

# -------------- 直近（過去へ遡る） --------------
def fetch_most_recent(point: str, max_days_back: int = 30, status=None):
    """
    今日→昨日→… と最大 max_days_back 日遡って、
    各日について CANDIDATE_HOURS を遅いものからチェックし、
    最初に見つかった回を返す。
    """
    tz = ZoneInfo("Asia/Tokyo")
    now = datetime.now(tz)
    for d in range(0, max_days_back + 1):
        day = (now - timedelta(days=d))
        y, m, dy = day.year, day.month, day.day
        hours = CANDIDATE_HOURS[:]
        if d == 0:
            hours = [h for h in hours if h <= now.hour] or hours  # 当日で未来の時刻は除外
        for h in hours:
            if status is not None:
                status.value = f"確認中: {y:04d}-{m:02d}-{dy:02d} {h:02d}時"
            df = fetch_one(point, y, m, dy, h)
            if df is not None:
                return datetime(y, m, dy, h, tzinfo=tz), df
    return None, None

# -------------- UI --------------
status = W.HTML(value="地点リスト取得中…")
display(status)

try:
    station_options = fetch_station_list()
    status.value = f"地点候補: {len(station_options)}件 取得。『直近のデータ』を探します。"
except Exception as e:
    station_options = [("（取得失敗）", "47807")]
    status.value = f"<b style='color:#c00'>地点取得に失敗:</b> {e}"

dd_station = W.Dropdown(options=station_options, description="地点:", layout=W.Layout(width="650px"))
days_back = W.IntSlider(value=30, min=1, max=90, step=1, description="直近探索（日）:", continuous_update=False)
btn_fetch = W.Button(description="直近データを取得", button_style="primary")
out = W.Output()
csv_name = W.Text(value="", description="CSV名:", placeholder="未指定なら自動命名")
btn_save = W.Button(description="CSV保存")
save_status = W.HTML()

_last = {"dt": None, "df": None, "point": None}

def on_fetch(_):
    with out:
        clear_output()
        save_status.value = ""
        btn_fetch.disabled = True
        try:
            p = dd_station.value
            status.value = "直近データを探索しています…"
            dt, df = fetch_most_recent(p, max_days_back=days_back.value, status=status)
            if df is None:
                status.value = "<b style='color:#c00'>直近データが見つかりませんでした。</b>"
                return
            _last.update({"dt": dt, "df": df, "point": p})
            jst = dt.strftime("%Y-%m-%d %H:%M JST")
            status.value = f"<b>取得成功:</b> {jst} / 取得元: {df.attrs.get('source','')}"
            display(df)
            if not csv_name.value:
                csv_name.value = f"upper_{p}_{dt.strftime('%Y%m%d%H')}.csv"
        except Exception as e:
            status.value = f"<b style='color:#c00'>エラー:</b> {e}"
        finally:
            btn_fetch.disabled = False

def on_save(_):
    if _last["df"] is None:
        save_status.value = "<b style='color:#c00'>先に『直近データを取得』してください。</b>"
        return
    name = (csv_name.value or f"upper_{_last['point']}_{_last['dt'].strftime('%Y%m%d%H')}.csv").strip()
    _last["df"].to_csv(name, index=False, encoding="utf-8-sig")
    save_status.value = f"<b>保存しました:</b> {name}"

btn_fetch.on_click(on_fetch)
btn_save.on_click(on_save)

display(W.VBox([
    dd_station,
    days_back,
    W.HBox([btn_fetch]),
    out,
    W.HBox([csv_name, btn_save]),
    save_status
]))

HTML(value='地点リスト取得中…')